In [9]:
import time
import random
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
model = Sequential()
model.add(Dense(20, input_dim=244, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [3]:
serialize_df = pd.read_csv("C:\\Users\\giova\\OneDrive\\Desktop\\serialized_data.csv")
serialize_data = serialize_df.to_numpy()
np.random.seed(seed=42)
np.random.shuffle(serialize_data)
splid_val = int(len(serialize_df)*0.80)
train_serial_data = serialize_data[:splid_val]
test_serial_data = serialize_data[splid_val:]
x_test = test_serial_data[:, 1:]
y_test = test_serial_data[:, 0]
serialize_data_X = train_serial_data[:, 1:]
serialize_data_Y = train_serial_data[:, 0]

In [4]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.002,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-8,), metrics=['accuracy'])

history = model.fit(serialize_data_X, serialize_data_Y, epochs=40, batch_size=128)

score, acc = model.evaluate(x_test, y_test, batch_size=128)
print(model.summary())
print('Test score:', score)
print('Test accuracy:', acc)

Epoch 1/40
129/129 [==============================] - 1s 1ms/step - loss: 0.1812 - accuracy: 0.7184
Epoch 2/40
129/129 [==============================] - 0s 1ms/step - loss: 0.1332 - accuracy: 0.8017
Epoch 3/40
129/129 [==============================] - 0s 1ms/step - loss: 0.1148 - accuracy: 0.8354
Epoch 4/40
129/129 [==============================] - 0s 977us/step - loss: 0.1002 - accuracy: 0.8545
Epoch 5/40
129/129 [==============================] - 0s 854us/step - loss: 0.0891 - accuracy: 0.8750
Epoch 6/40
129/129 [==============================] - 0s 1ms/step - loss: 0.0793 - accuracy: 0.8865
Epoch 7/40
129/129 [==============================] - 0s 977us/step - loss: 0.0734 - accuracy: 0.8940
Epoch 8/40
129/129 [==============================] - 0s 1ms/step - loss: 0.0683 - accuracy: 0.9027
Epoch 9/40
129/129 [==============================] - 0s 1ms/step - loss: 0.0630 - accuracy: 0.9097
Epoch 10/40
129/129 [==============================] - 0s 939us/step - loss: 0.0604 - accuracy

In [65]:
times = []
res1 = []
print(len(x_test)//4)
for i in range(len(x_test)//4):
    start_time = time.time()
    #y_pred = model.predict(serialize_data[random.randint(0, 20495)][1:].reshape(1, 244),verbose = 0)
    y_pred = model.predict(x_test[i].reshape(1,244),verbose = 0)
    times.append(time.time() - start_time)
    res1.append(y_pred)
print(np.mean(times))

1025
0.0360576624986602


In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [50]:
rf = RandomForestRegressor(n_estimators = 10, random_state = 42)
rf.fit(serialize_data_X, serialize_data_Y);

In [66]:
# Use the forest's predict method on the test data
predictions = rf.predict(x_test)
print(predictions)

# Calculate and display accuracy
mse = mean_squared_error(y_test, predictions)
print('Mse:', round(mse, 2), '%.')

[0.9        0.         1.         ... 0.         1.         0.45202361]
Mse: 0.06 %.


In [67]:
times = []
res2 = []
for i in range(len(x_test)//4):
    start_time = time.time()
    y_pred = rf.predict(x_test[i].reshape(1,244))
    times.append(time.time() - start_time)
    res2.append(y_pred)
print(np.mean(times))

0.0005089694697682451


In [55]:
res1 = model.predict(x_test,verbose = 0)

In [72]:
print(np.linalg.norm(y_test[:len(x_test)//4] - np.concatenate(res1)))
print(np.linalg.norm(y_test[:len(x_test)//4] - np.concatenate(res2)))

687.3874509277093
7.6184423475927145


In [75]:
print(np.concatenate(res1).shape)
print(y_test.shape)
print(np.concatenate(res2).shape)

(1025, 1)
(4100,)
(1025,)


In [76]:
print(np.concatenate(res1))
print(y_test)
print(np.concatenate(res2))

[[9.7528905e-01]
 [4.0252408e-07]
 [9.9401492e-01]
 ...
 [3.4742981e-02]
 [3.7841748e-02]
 [9.9383467e-01]]
[1 0 1 ... 0 1 1]
[0.9        0.         1.         ... 0.         0.14444444 1.        ]
